In [130]:
import requests
from bs4 import BeautifulSoup as bs
last = 34129053
url = 'https://www.wissports.net/stats/league_instance/148015?subseason=763824&tab=league_instance_team_stats&tool=4618648'
#data = requests.get(url)
first = 33030961
#first = 34129052
#params = {
    #"id" : "#tab_gs3_game_stats_content",
 #   "class": "tab+menu-tab z6 loaded"
#}
my_range = range(first, last + 1)
header = {'class' : 'basketball minimal user_mode gameShow mobile'}
#for game in my_range:
        #this_url = url.format(game)
        #data = requests.get(this_url)
data = requests.get(url)
if data.status_code == 200:
        with open(r'C:\Users\Jake\Documents\Data\test.html', 'w+') as f:
                f.write(data.text)

#root = html.fromstring(page.content)
# <div id="GameShowContent" class=" game_show_nextgen">    

In [2]:
import re
from bs4 import BeautifulSoup as bs
import requests
url = 'https://www.wissports.net/game/show/33030957?subseason=763824'
data = requests.get(url)



In [125]:
import os 
full_game = []
directory = r'C:\Users\Jake\Documents\Data'
for file in os.listdir(directory):
        game_id = 1
        f = os.path.join(directory, file)
        if os.path.isfile(f):
                with open(f) as file:
                        soup = bs(file, 'html5lib')
                        games = soup.find_all('table')
                        scores = soup.find_all('tbody')
                        scores = str(scores).split('\n')
                        my_teams = []
                        first_half = []
                        second_half = []
                        field_goals = []
                        ft = []
                        threes = []
                        assists =[]
                        rebounds = []
                        turnovers = []
                        steals = []
                        blocks = []
                        fouls = []
                        game_id_list = []
                        pattern = '>(.*?)<'
                        for line in scores:
                                if (('"team">')) in line:
                                        my_teams.append(re.search(pattern, line).group(1))
                                elif (("pts_1")) in line:
                                        first_half.append(re.search(pattern, line).group(1))
                                elif (("pts_2")) in line:
                                        second_half.append(re.search(pattern, line).group(1))
                        final = soup.find_all('thead')
                        final = str(final).split('</th>')
                        for line in final:
                                if (('<th title="Field Goals Made">\n')) in line:
                                        field_goals.append(line.split('\n')[2].strip())
                                elif (('<th title="Free Throws Made">\n')) in line:
                                        ft.append(line.split('\n')[2].strip())
                                elif (('<th title="Three Point Made">\n')) in line:
                                        threes.append(line.split('\n')[2].strip())
                                elif (('<th title="Assists">\n')) in line:
                                        assists.append(line.split('\n')[2].strip())
                                elif (('<th title="Turnovers">\n')) in line:
                                        turnovers.append(line.split('\n')[2].strip())
                                elif (('<th title="Steals">\n')) in line:
                                        steals.append(line.split('\n')[2].strip())
                                elif (('<th title="Blocks">\n')) in line:
                                        blocks.append(line.split('\n')[2].strip())
                                elif (('<th title="Fouls">\n')) in line:
                                        fouls.append(line.split('\n')[2].strip())
                                elif (('<th title="Total Rebounds">\n')) in line:
                                        rebounds.append(line.split('\n')[2].strip())
                        if len(my_teams) > 0:
                                final = list(zip(my_teams, first_half, second_half, field_goals, ft, threes, assists, turnovers, steals, blocks, fouls, rebounds))
                                full_game.append(final)



In [126]:
import pandas as pd
import numpy as np
game_full = pd.DataFrame(full_game)
game_full[0] = game_full[0].apply(lambda x: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0) if x==None else x)
game_full[1] = game_full[1].apply(lambda x: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0) if x==None else x)
split_1 = pd.DataFrame(game_full[0].tolist(), columns = ['Team', 'First_Half_Score', 'Second_Half_Score', 'Field_Goals', 'Free_Throws','Threes', 'Assists', 'Turnovers', 'Steals', 'Blocks', 'Fouls', 'Rebounds'])
split_2 = pd.DataFrame(game_full[1].tolist(), columns = ['Team', 'First_Half_Score', 'Second_Half_Score', 'Field_Goals', 'Free_Throws','Threes', 'Assists', 'Turnovers', 'Steals', 'Blocks', 'Fouls', 'Rebounds'])
split_1['Game_Id'] = split_1.index 
split_2['Game_Id'] = split_2.index 
team_games = pd.concat([split_1, split_2], axis = 0)
fg = team_games['Field_Goals'].str.split('-', n = 1, expand = True)
ft = team_games['Free_Throws'].str.split('-', n = 1, expand = True)
threes = team_games['Threes'].str.split('-', n = 1, expand = True)
team_games['FGM'] = fg[0]
team_games['FGA'] = fg[1]
team_games['FTM'] = ft[0]
team_games['FTA'] = ft[1]
team_games['3FGM'] = threes[0]
team_games['3FGA'] = threes[1]
team_games = team_games.drop(columns = ['Field_Goals', 'Free_Throws', 'Threes'])


In [131]:
team_games = team_games.dropna()
team_games = team_games.drop_duplicates()
team_games.to_csv('each_team_game.csv', index = False)

In [132]:
full_game = pd.merge(split_1, split_2, on = 'Game_Id')
full_game.to_csv('full_games.csv', index = False)

In [133]:
team_games

,Team,First_Half_Score,Second_Half_Score,Assists,Turnovers,Steals,Blocks,Fouls,Rebounds,Game_Id,FGM,FGA,FTM,FTA,3FGM,3FGA
0,Lac Courte Oreilles,0,57,3,24,4,2,0,36,0,8,32,4,20,1,3
1,Lac Courte Oreilles,14,19,20,10,16,1,0,37,1,26,67,8,10,6,25
2,Lac Courte Oreilles,6,14,0,0,0,0,0,0,2,0,0,0,0,0,0
3,Lac Courte Oreilles,16,24,14,0,18,5,0,33,3,15,44,9,12,8,22
4,Lac Courte Oreilles,16,13,11,0,22,5,0,45,4,20,51,14,20,6,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1887,Oneida Nation,29,33,10,11,8,0,16,30,1887,22,46,16,25,2,13
1888,Oneida Nation,26,33,12,21,7,2,26,32,1888,22,55,10,13,5,16
1889,Oneida Nation,32,19,6,14,12,0,14,31,1889,14,43,23,26,0,4
1892,Oneida Nation,0,70,15,12,17,5,16,44,1892,29,65,6,12,6,19


In [4]:
import re
from bs4 import BeautifulSoup as bs
import requests
url = 'https://www.wissports.net/game/show/33500097?subseason=750990&referrer=6395643'
data = requests.get(url)
if data.status_code == 200:
        with open(r'C:\Users\Jake\Documents\Data\test2.html', 'w+') as f:
                f.write(data.text)

In [5]:
soup = bs(open(r'C:\Users\Jake\Documents\Data\test2.html'), 'html5lib')
games = soup.find_all('table')

In [77]:
games = soup.find_all('table')
scores = soup.find_all('tbody')
scores = str(scores).split('\n')
scores

['[<tbody>',
 '    <tr class="box-score-border" id="lineScore-team-6406208">',
 '      <td class="team">Rice Lake</td>',
 '        <td class="pts_1">0</td>',
 '        <td class="pts_2">6</td>',
 '        <td class="pts_3">0</td>',
 '        <td class="pts_4">0</td>',
 '      <td class="away_team_score final_score score-final">6</td>',
 '    </tr>',
 '    <tr class="box-score-border" id="lineScore-team-6406180">',
 '      <td class="team">Pewaukee</td>',
 '        <td class="pts_1">8</td>',
 '        <td class="pts_2">0</td>',
 '        <td class="pts_3">0</td>',
 '        <td class="pts_4">7</td>',
 '      <td class="home_team_score final_score score-final">15</td>',
 '    </tr>',
 '  </tbody>, <tbody>',
 '          <tr class="odd">',
 '            <td class="jersey_no">7</td>',
 '            <td class="player_name" sorttable_customkey="FenskeCole">',
 '                <a class="" href="https://www.wissports.net/roster_players/46637112" title="View Cole Fenske\'s Player Profile">Cole 

In [88]:
first_quarter = []
second_quarter = []
third_quarter = []
fourth_quarter = []
my_teams = []
compeletions = []
attempts = []
yards = []
touchdowns = [] 
interceptions = []
qb_rat = []
avg_yards = []
receptions = []
s_tackles = []
t_tackles = []
sacks = []
ff = []
returns = []
avg_returns = []
punts = []
yp_punt = []
fgm = []
xpm = []
xpa = []
pattern = '>(.*?)<'
for line in scores:
    if (('"team">')) in line:
        my_teams.append(re.search(pattern, line).group(1))
    elif (("pts_1")) in line:
        first_quarter.append(re.search(pattern, line).group(1))
    elif (("pts_2")) in line:
        second_quarter.append(re.search(pattern, line).group(1))
    elif (("pts_3")) in line:
        third_quarter.append(re.search(pattern, line).group(1))
    elif (("pts_4")) in line:
        fourth_quarter.append(re.search(pattern, line).group(1))
final = soup.find_all('thead')
final = str(final).split('</th>')
print(final)
for line in final:
    if (('<th title="Completions">\n')) in line:
        if len(compeletions) < 2:
            compeletions.append(line.split('\n')[2].strip())
    elif (('<th title="Attempts">\n')) in line:
        if len(attempts) < 4:
            attempts.append(line.split('\n')[2].strip())
    elif (('<th title="Yards">\n')) in line:
        if len(yards) < 10:
            yards.append(line.split('\n')[2].strip())
    elif (('<th title="Touchdowns">\n')) in line:
        if len(touchdowns) < 10:
            touchdowns.append(line.split('\n')[2].strip())
    elif (('<th title="Interceptions">\n')) in line:
        if len(interceptions) < 4:
            interceptions.append(line.split('\n')[2].strip())
    elif (('<th title="QB Rating">\n')) in line:
        if len(qb_rat) < 2:
            qb_rat.append(line.split('\n')[2].strip())
    elif (('<th title="Average Yards">\n')) in line:
        if len(avg_yards) < 4:
            avg_yards.append(line.split('\n')[2].strip())
    elif (('<th title="Receptions">\n')) in line:
        if len(receptions) < 2:
            receptions.append(line.split('\n')[2].strip())
    elif (('<th title="Solo Tackles">\n')) in line:
        if len(s_tackles) < 2:
            s_tackles.append(line.split('\n')[2].strip())
    elif (('<th title="Total Tackles">\n')) in line:
        if len(t_tackles) < 2:
            t_tackles.append(line.split('\n')[2].strip())
    elif (('<th title="Sacks">\n')) in line:
        if len(sacks) < 2:
            sacks.append(line.split('\n')[2].strip())
    elif (('<th title="Forced Fumbles">\n')) in line:
        if len(ff) < 2:
            ff.append(line.split('\n')[2].strip())
    elif (('<th title="Returns">\n')) in line:
        if len(returns) < 4:
            returns.append(line.split('\n')[2].strip())
    elif (('<th title="Yards/Return">\n')) in line:
        if len(avg_returns) < 4:
            avg_returns.append(line.split('\n')[2].strip())
    elif (('<th title="Punts">\n')) in line:
        if len(punts) < 2:
            punts.append(line.split('\n')[2].strip())  
    elif (('<th title="Yards Per Punt">\n')) in line:
        if len(yp_punt) < 2:
            yp_punt.append(line.split('\n')[2].strip()) 
    elif (('<th title="Field Goals Made">\n')) in line:
        if len(fgm) < 2:
            fgm.append(line.split('\n')[2].strip())  
    elif (('<th title="Extra Point Attempts">\n')) in line:
        if len(xpm) < 2:
            xpm.append(line.split('\n')[2].strip())  
    elif (('<th title="Extra Points Made">\n')) in line:
        if len(xpa) < 2:
            xpa.append(line.split('\n')[2].strip())           

['[<thead>\n    <tr class="periods">\n      <th class="team">', '\n        <th title="1st Period">1', '\n        <th title="2nd Period">2', '\n        <th title="3rd Period">3', '\n        <th title="4th Period">4', '\n        <th class="total">T', '\n    </tr>\n  </thead>, <thead>\n      <tr>\n        <th title="Jersey Number">#', '\n        <th class="player_name" title="Player Name">Name', '\n          <th title="Completions">Comp', '\n          <th title="Attempts">Att', '\n          <th title="Yards">Yds', '\n          <th title="Touchdowns">TD', '\n          <th title="Interceptions">Int', '\n          <th title="QB Rating">Rate', '\n      </tr>\n    </thead>, <thead>\n      <tr class="even totals">\n        <th>', '\n        <th class="player_name">Totals', '\n          <th title="Completions">\n            3\n          ', '\n          <th title="Attempts">\n            9\n          ', '\n          <th title="Yards">\n            32\n          ', '\n          <th title="Touchdow

In [89]:
full_game = []
rushing_avg = avg_yards[0:1]
reciving_avg = avg_yards[2:3]
passing_attempts = attempts[0:1]
rushing_attempts = attempts[2:3]
passing_yards = yards[0:1]
rushing_yards = yards[2:3]
recieving_yards = yards[4:5]
kickoff_return_yards = yards[6:7]
punt_yds = yards[8:9]
passing_tds = touchdowns[0:1]
rushing_tds = touchdowns[2:3]
recieving_tds = touchdowns[4:5]
punt_return_tds = touchdowns[6:7]
kickoff_tds = touchdowns[8:9]
passing_interceptions = interceptions[0:1]
defensive_interceptions = interceptions[2:3]
punt_returns = returns[0:1]
kickoff_returns = returns[2:3]
avg_punt_return = avg_returns[0:1]
avg_kickoff_return = avg_returns[2:3]
if len(my_teams) > 0:
    final = list(zip(my_teams, first_quarter, second_quarter, third_quarter, fourth_quarter, compeletions, passing_attempts, passing_tds, qb_rat, passing_yards, 
    passing_interceptions, 
                     rushing_attempts, rushing_tds, rushing_yards,rushing_avg,  receptions, recieving_yards, recieving_tds, reciving_avg, s_tackles, t_tackles, sacks,
                      ff, defensive_interceptions,
                     punt_returns, avg_punt_return, punt_return_tds,  kickoff_returns, kickoff_tds, kickoff_return_yards, avg_kickoff_return, punts, punt_yds, yp_punt, 
                     fgm, xpa, xpm))
    full_game.append(final)

avg_punt_return

['5.0']

In [94]:
import pandas as pd
import numpy as np
game_full = pd.DataFrame(full_game)
game_full[1].tolist()

KeyError: 1

In [93]:
split_1 = pd.DataFrame(game_full[0].tolist(), columns = ['Team', 'First_Quarter', 'Second_Quarter',
                                                         'Third_Quarter', 'Fourth_Quarter','Passing_Completions', 'Passing_Attempts', 'Passing_Tds', 
                                                         'QB_Rating', 'Passing_Yards', 'Passing_Interceptions', 'Rushing_Attempts', 'Rushing_Tds', 'Rushing_Yards', 'Rushing_Avg', 
                                                         'Receptions', 'Recieving_Yards', 'Recieving_Tds', 'Recieving_Avg', 'Solo_Tackles', 'Team_Tackles', 'Sacks', 'Forced_Fumbles',
                                                         'Defensive_Interceptions', 'Punt_Returns', 'Avg_Punt_Returns', 'Punt_Return_Tds', 'Kickoff_Returns', 'Kickoff_Tds',
                                                         'Kickoff_Return_Yards', 'Avg_Kickoff_Return', 'Punts', 'Punt_yds', 'Yp-Punt', 'FGM', 'XPA', 'XPM'])
#split_2 = pd.DataFrame(game_full[1].tolist(), columns = ['Team', 'First_Quarter', 'Second_Quarter',
                                             #            'Third_Quarter', 'Fourth_Quarter','Passing_Completions', 'Passing_Attempts', 'Passing_Tds', 
                                              #           'QB_Rating', 'Passing_Yards', 'Passing_Interceptions', 'Rushing_Attempts', 'Rushing_Tds', 'Rushing_Yards', 'Rushing_Avg', 
                                               #          'Receptions', 'Recieving_Yards', 'Recieving_Tds', 'Recieving_Avg', 'Solo_Tackles', 'Team_Tackles', 'Sacks', 'Forced_Fumbles',
                                                #         'Defensive_Interceptions', 'Punt_Returns', 'Avg_Punt_Returns', 'Punt_Return_Tds', 'Kickoff_Returns', 'Kickoff_Tds',
                                                 #        'Kickoff_Return_Yards', 'Avg_Kickoff_Return', 'Punts', 'Punt_yds', 'Yp-Punt', 'FGM', 'XPA', 'XPM'])
split_1['Game_Id'] = split_1.index 
split_1
#split_2['Game_Id'] = split_2.index 

,Team,First_Quarter,Second_Quarter,Third_Quarter,Fourth_Quarter,Passing_Completions,Passing_Attempts,Passing_Tds,QB_Rating,Passing_Yards,...,Kickoff_Tds,Kickoff_Return_Yards,Avg_Kickoff_Return,Punts,Punt_yds,Yp-Punt,FGM,XPA,XPM,Game_Id
0,Rice Lake,0,6,0,0,3,9,0,5.1,32,...,0,44,16.0,1,43,43.0,0,0,0,0
